In [8]:
import pandas as pd
import glob, gzip
import os
import mwparserfromhell
import re

In [9]:
def process_dump(dump,  path):
    global main
    global seealso
    global further
    a = True
    for page in dump:
        if page.namespace == 0:
            for revision in page:
                m = []
                s = []
                f = []
                try:
                    text = revision.text
                    text = re.sub('{{!}}|{{¡}}|\]\] e \[\[|\]\], \[\[', '|', text)
                except:
                    print(text)
                for m_i in main:
                    try:
                        m_ = re.findall(r'{{\s?%s\s?\|.*?}}' % m_i, text)
                        m_ = [i.strip().split('#')[0].strip('[]') for j in m_ for i in j.strip('{{}} ').split('|')[1:] if '=' not in i]
                        m = m+m_
                    except:
                        pass
#                 for s_i in seealso:
#                     try:
#                         s_ = re.findall(r'{{\s?%s\s?\|.*?}}' % s_i, text)
#                         s_ = [i.strip().split('#')[0].strip('[]') for j in s_ for i in j.strip('{{}} ').split('|')[1:] if '=' not in i]
#                         s = s+s_
#                     except:
#                         pass
#                 for f_i in further:
#                     try:
#                         f_ = re.findall(r'{{\s?%s\s?\|.*?}}' % f_i, text)
#                         f_ = [i.strip().split('#')[0].strip('[]') for j in f_ for i in j.strip('{{}} ').split('|')[1:] if '=' not in i]
#                         f = f + f_
#                     except:
#                         pass
                yield revision.page.title, list(set(m)), list(set(s)), list(set(f))

In [ ]:
import mwxml
import json
import csv
import time

date = "20201120"

mains = {'en':['Main', 'main', 'Mainarticle', 'mainarticle', 'Readmain', 'readmain', 'Seemain', 'seemain', 
               'See main', 'see main', 'Main articles', 'main articles', 'Hurricane main', 'hurricane main',
               'Main1', 'main1', 'Main2', 'main2', 'Comprehensive', 'comprehensive', 'MAIN'],
         'es':['AP', 'ap', 'Main', 'main', 'Catmas', 'catmas', 'Cp', 'cp', 'Catmain', 'catmain', 'CP', 
               'Catmás', 'catmás', 'Principal', 'principal', 'Catmore', 'catmore', 
               'Artículo principal', 'artículo principal', 'Cat main', 'cat main'],
         'fr':['Article détaillé', 'article détaillé', 'Détails', 'détails', 
               'Voir', 'voir', 'Détail', 'détail', 'Loupe', 'loupe'], 
         'de':['Hauptartikel', 'hauptartikel', 'Hauptseite', 'hauptseite', 'Main', 'main'], 
         'ar':['مفصلة','رئيسي', 'Main', 'مقال رئيسي', 'رئيسية', 'Main article', 'main article'
               'مقالة تفصيلية', 'مقال تفصيلي', 'مقالة رئيسية', 'مقالة مفصلة', 'مقالة منفصلة'],
         'pt':['Artigo principal', 'artigo principal', 'Ver artigo principal', 'ver artigo principal', 
               'Principal', 'principal', 'Ver também artigo principal', 'ver também artigo principal', 
               'Main', 'main', 'Detalhes', 'detalhes', 'Artigoprincipal', 'artigoprincipal', 'AP', 'ap', 'Details', 'details',
               'Ver artigo', 'ver artigo', 'Artigo Principal'],
         'zh':['Main', 'main', '细节', 'Main articles', 'main articles', '主条目', 'Hurricane main', 'hurricane main', 
               '条目', '主', '頁面', '主頁面', '主頁', '主題目'], 
         'it':['Vedi anche','vedi anche', 'Main', 'main', 'Per approfondire', 'per approfondire', 
               'See also', 'see also', 'Further', 'further'], 
         'ru':['Основная статья', 'основная статья', 
               'Основная статья категории', 'основная статья категории',
               'Catmore', 'catmore', 'Catmain', 'catmain', 'Main articles', 'main articles', 
               'Основная статья раздела', 'основная статья раздела', 
               'Главное', 'главное', 'Главная', 'главная', 'CatMain',  
               'Основная', 'основная', 'Maincat'],
         'ko':['본문', 'Main', 'main', '원문', 'Main article', 'main article', 'Main2', 'main2'], 
         'ja':['Main', 'main', 'Main article', 'main article', 'Further', 'further', 'Mainarticle', 'mainarticle', 
               'Details', 'details', '本文', 'More', 'more', 'Further information', 'further information', '詳細記事']}

# seealsos = {'en':['See also', 'Seealso', 'Also', 'See Also', 'Looking', 'Related articles', 
#                   'Related article', 'See-Also', 'See-also', 'SeeAlso', 'Looking for'], 
#             'es':['VT', 'Ver', 'Vt', 'Véase también', 'Seealso', 'Véase', 'See also', 'Más información', 'Further', 'Futher'],
#             'fr':['Article connexe', 'Voir aussi', 'Articles connexes', 'See also', 'Article lié'],
#             'de':['Siehe auch'],
#             'ar':['أيضا', 'انظر أيضاً', 'أنظر أيضا', 'See also', 
#                   'Seealso', 'طالع أيضاً', 'See', 'طالع', 'طالع ايضا', 'طالع أيضًا', 'طالع أيضا'],
#             'pt':['VT','Outro artigo', 'Ver outro artigo', 'Veja', 'Anexo', 'Ver anexo', 
#                   'Seealso', 'Ver página anexa', 'Vertambém', 'See also', 'Vt'],
#             'zh':['See also', 'Seealso', '参看', '参见', '另见', 'SeeAlso', 'See Also', 'Also', '參見', '參看'],
#             'it':[],
#             'ru':['См. также', 'Также', 'See also', 'Seealso', 'Addon', 
#                   'Also', 'Смотри также', 'Смотрите также'], 
#             'ko':['참고', '참조', 'Seealso', 'See also', 'See', 'Also'], 
#             'ja':['See also', 'Seealso', 'Vedi anche', 'Further2', 'Also', '関連記事']}

# furthers = {'en':['further', 'Details', 'See details', 'Seedetails', 'Seesubarticle', 
#                   'More', 'Detail', 'Details more', 'Moredetails', 'More information', 'See'],
#             'es':[], 
#             'fr':['Lire la suite'], 
#             'de':[], 
#             'ar':['معلومات أكثر', 'Further', 'معلومات اكثر'], 
#             'pt':['Mais informações', 'Further', 'Mais', 'Catmore1', 'See'], 
#             'zh':['Further', 'Details', 'See', 'More', '相關條目', 'Detail', '见', '更多资料', 
#                   'Further information', '更多资讯', 'More information'], 
#             'it':[], 
#             'ru':['Подробно', 'Further'], 
#             'ko':['추가 정보', 'Further', 'Further information'], 
#             'ja':['See']}

start = time.time()
langs = ['pt','ko','ja','en','es','fr','de','ar','it','zh','ru']

for lang in langs:
    paths = glob.glob('../common/data/20201120/%swiki-20201120-pages-meta-current.xml.bz2' % lang)
                      
    print(paths)
    print(lang)
    m_dict = dict()
    s_dict = dict()
    f_dict = dict()
    main = mains[lang]
#     seealso = seealsos[lang]
#     further = furthers[lang]
    for parent, m, s, f in mwxml.map(process_dump, paths, threads = 16): # what meaning for threads?
        if len(m) != 0:
            m_ = [re.sub('[\[\]#\<\>\|\{\}]','',i) for i in m]
            m_dict[parent] = m_
#         if len(s) != 0:
#             s_ = [re.sub('[\[\]#\<\>\|\{\}]','',i) for i in s]
#             s_dict[parent] = s
#         if len(f) != 0:
#             f_ = [re.sub('[\[\]#\<\>\|\{\}]','',i) for i in f]
#             f_dict[parent] = f
    with open("./kcc2021/%s_m.json" % lang, 'w', encoding='utf-8') as f:
        f.write(json.dumps(m_dict, ensure_ascii=False))
#     with open("./adjacency_results2/%s_s.json" % lang, 'w', encoding='utf-8') as f:
#         f.write(json.dumps(s_dict, ensure_ascii=False))
#     with open("./adjacency_results2/%s_f.json" % lang, 'w', encoding='utf-8') as f:
#         f.write(json.dumps(f_dict, ensure_ascii=False))

    print(time.time()-start)
    start = time.time()
f.close()
        
        

['../common/data/20201120/ptwiki-20201120-pages-meta-current.xml.bz2']
pt
1406.5091202259064
['../common/data/20201120/kowiki-20201120-pages-meta-current.xml.bz2']
ko
544.402672290802
['../common/data/20201120/jawiki-20201120-pages-meta-current.xml.bz2']
ja
1408.243817806244
['../common/data/20201120/enwiki-20201120-pages-meta-current.xml.bz2']
en
None
13840.144894599915
['../common/data/20201120/eswiki-20201120-pages-meta-current.xml.bz2']
es
2176.757666826248
['../common/data/20201120/frwiki-20201120-pages-meta-current.xml.bz2']
fr
3058.49263381958
['../common/data/20201120/dewiki-20201120-pages-meta-current.xml.bz2']
de
2708.392370700836
['../common/data/20201120/arwiki-20201120-pages-meta-current.xml.bz2']
ar


Namespace id conflict detected.  <title>=مستخدم:منصورالواقدي/مقالات, <namespace>=105, mapped_namespace=2


1553.7821242809296
['../common/data/20201120/itwiki-20201120-pages-meta-current.xml.bz2']
it
1956.556280374527
['../common/data/20201120/zhwiki-20201120-pages-meta-current.xml.bz2']
zh
None
1588.3190937042236
['../common/data/20201120/ruwiki-20201120-pages-meta-current.xml.bz2']
ru


In [5]:
print(text)

NameError: name 'text' is not defined